In [15]:
import pandas as pd
import glob
import numpy as np
from decimal import *
  
path = "C:\\Users\\AROLMCAC\\Desktop\\Test python\\Test"
  
filenames = glob.glob(path + "\*.xlsx")
print('File names:', filenames)
  
finalexcelsheet = pd.DataFrame()
  
for file in filenames:
  
    df = pd.concat(pd.read_excel(file, sheet_name=None), ignore_index=True, sort=False)
  
    finalexcelsheet = finalexcelsheet.append(df, ignore_index=True)
    

FINALDF = finalexcelsheet.fillna(0)
FINALDF["Compte"] = FINALDF["Compte"].apply(np.int64)

FINALDF1 = FINALDF.drop(columns=['Total du 01/01/2021 au 01/01/2021 : Débit','Total du 01/01/2021 au 01/01/2021 : Crédit','Total du 01/01/2022 au 30/06/2022 : Débit','Total du 01/01/2022 au 30/06/2022 : Crédit','Total du 01/01/2022 au 30/06/2022 : Crédit','Total du 01/01/2021 au 31/12/2021 : Débit','Total du 01/01/2021 au 31/12/2021 : Crédit']) 
print('Final result:')

FINALDF1['LEFT'] = FINALDF1['Compte'].astype(str).str[:1]

#print(FINALDF2.dtypes)
FINALDF1['Solde_2020'] = FINALDF1["Solde du 01/01/2021 au 01/01/2021 : Débit"] - FINALDF1["Solde du 01/01/2021 au 01/01/2021 : Crédit"]
FINALDF1['Solde_2021'] = FINALDF1["Solde du 01/01/2021 au 31/12/2021 : Débit"] - FINALDF1["Solde du 01/01/2021 au 31/12/2021 : Crédit"]
FINALDF1['Solde_2022'] = FINALDF1["Solde du 01/01/2022 au 30/06/2022 : Débit"] - FINALDF1["Solde du 01/01/2022 au 30/06/2022 : Crédit"]

FINALDF1['Solde_2020'] = FINALDF1['Solde_2020'].astype(int)
FINALDF1['Solde_2021'] = FINALDF1['Solde_2021'].astype(int)
FINALDF1['Solde_2022'] = FINALDF1['Solde_2022'].astype(int)

FINALDF2 = FINALDF1.groupby( ['LEFT'])['Solde_2020','Solde_2021','Solde_2022'].sum().reset_index()

FINALDF3 = FINALDF1.groupby( ['Compte'])['Solde_2020','Solde_2021','Solde_2022'].sum().reset_index()

#POSTINGS----------------------------------------------------------------------------------------------------------------------

path = "C:\\Users\\AROLMCAC\\Desktop\\Test python\\Accounting_data"

filenames = glob.glob(path + "\*.xlsx")
print('File names:', filenames)
  
finalexcelsheet = pd.DataFrame()
  
for file in filenames:
  
    df = pd.concat(pd.read_excel(file, sheet_name=None), ignore_index=True, sort=False)
  
    finalexcelsheet = finalexcelsheet.append(df, ignore_index=True)
    
finalexcelsheet['LEFT'] = finalexcelsheet['GENERAL'].astype(str).str[:1]


finalexcelsheet = finalexcelsheet[finalexcelsheet['JAL'] != "ANO"]

finalexcelsheet['NUM'] = finalexcelsheet['NUM'].astype(int)
finalexcelsheet['NUM'] = finalexcelsheet['NUM'].astype(str)

finalexcelsheet['CONCATENATE'] = (finalexcelsheet['DATE'].astype(str).str[:4])+finalexcelsheet['JAL']+finalexcelsheet['NUM']
finalexcelsheet['AMOUNT'] = finalexcelsheet['DEBIT']-finalexcelsheet['CREDIT']
finalexcelsheet['AMOUNT'] = finalexcelsheet['AMOUNT'].astype(int)
FINALPOSTINGS = finalexcelsheet[[ 'LEFT','DEBIT','CREDIT','AMOUNT']]


FINALPOSTINGS1 = FINALPOSTINGS.groupby( ['LEFT'])['DEBIT','CREDIT','AMOUNT'].sum().reset_index()

FINALPOSTINGS1['DEBIT'] = FINALPOSTINGS1['DEBIT'].astype(int)
FINALPOSTINGS1['CREDIT'] = FINALPOSTINGS1['CREDIT'].astype(int)
FINALPOSTINGS1['AMOUNT'] = FINALPOSTINGS1['AMOUNT'].astype(int)

#TEST------------------------------------------------------------------------------------------------------------------------

Result = pd.merge(FINALDF2, FINALPOSTINGS1, on="LEFT")

Result1 = Result[[ 'LEFT','Solde_2020','DEBIT','CREDIT','Solde_2022']]

Result1.rename(columns = {'Solde_2020':'Beginning Balance'}, inplace = True)
Result1.rename(columns = {'Solde_2022':'Ending Balance by Entity'}, inplace = True)
Result1.rename(columns = {'DEBIT':'Debit postings'}, inplace = True)
Result1.rename(columns = {'CREDIT':'Credit postings'}, inplace = True)

Result1['Ending Balance by FFP'] =  Result1['Beginning Balance'] + Result1['Debit postings'] - Result1['Credit postings']

Result1['Dif'] = Result1['Ending Balance by FFP'] - Result1['Ending Balance by Entity']

Result1['Credit postings'] =  - Result1['Credit postings']

Result1['Nature'] = np.where(Result1['LEFT'] <= "5" , "BS","P&L" )

Result1 = Result1[[ 'LEFT','Nature','Beginning Balance','Debit postings','Credit postings','Ending Balance by FFP','Ending Balance by Entity','Dif']]

display(Result1)
 
#TEST_COMPTE-------------------------------------------------------------------------------------------------------------------


TEST_COMPTE = finalexcelsheet[[ 'LEFT','GENERAL','DEBIT','CREDIT','AMOUNT']]
TEST_COMPTE['GENERAL'] = TEST_COMPTE['GENERAL'].astype(int)
TEST_COMPTE.rename(columns = {'GENERAL':'Compte'}, inplace = True)


TB_COMPTE = FINALDF1[[ 'Compte','Solde_2020','Solde_2021','Solde_2022']]


ALL_TEST = pd.merge(TEST_COMPTE, TB_COMPTE, on= "Compte")

#ALL_TEST = ALL.groupby( ['Compte'])['DEBIT','CREDIT','Solde_2020','Solde_2021','Solde_2022'].sum().reset_index()

ALL_TEST.rename(columns = {'Solde_2020':'Beginning Balance'}, inplace = True)
ALL_TEST.rename(columns = {'Solde_2022':'Ending Balance by Entity'}, inplace = True)
ALL_TEST.rename(columns = {'DEBIT':'Debit postings'}, inplace = True)
ALL_TEST.rename(columns = {'CREDIT':'Credit postings'}, inplace = True)

ALL_TEST['Ending Balance by FFP'] =  ALL_TEST['Beginning Balance'] + ALL_TEST['Debit postings'] - ALL_TEST['Credit postings']

ALL_TEST['Dif'] = ALL_TEST['Ending Balance by FFP'] - ALL_TEST['Ending Balance by Entity']

ALL_TEST['Credit postings'] =  - ALL_TEST['Credit postings']

ALL_TEST['LEFT'] = ALL_TEST['Compte'].astype(str).str[:1]

ALL_TEST['Nature'] = np.where(ALL_TEST['LEFT'] <= "5" , "BS","P&L" )

ALL_TEST_FINAL = ALL_TEST[[ 'Compte','Nature','Beginning Balance','Debit postings','Credit postings','Ending Balance by FFP','Ending Balance by Entity','Dif']]

ALL_TEST_FINAL1 = ALL_TEST_FINAL.groupby( ['Compte'])['Nature','Beginning Balance','Debit postings','Credit postings','Ending Balance by FFP','Ending Balance by Entity','Dif'].sum().reset_index()
#----------------------------------------------------------------------------------------------------------------------
DF=pd.merge(TEST_COMPTE,TB_COMPTE, on='Compte', how='outer')
DF.rename(columns = {'Solde_2020':'Beginning Balance'}, inplace = True)
DF.rename(columns = {'Solde_2022':'Ending Balance by Entity'}, inplace = True)
DF.rename(columns = {'DEBIT':'Debit postings'}, inplace = True)
DF.rename(columns = {'CREDIT':'Credit postings'}, inplace = True)

DF['Ending Balance by FFP'] =  DF['Beginning Balance'] + DF['Debit postings'] - DF['Credit postings']

DF['Dif'] = DF['Ending Balance by FFP'] - DF['Ending Balance by Entity']

#DF['Credit postings'] =  - DF['Credit postings']

DF['LEFT'] = DF['Compte'].astype(str).str[:1]

DF['Nature'] = np.where(DF['LEFT'] <= "5" , "BS","P&L" )

ALL_TEST_FINAL3 = DF[[ 'Compte','Nature','Beginning Balance','Debit postings','Credit postings','Ending Balance by FFP','Ending Balance by Entity','Dif']]


#ALL_TEST_FINAL4 = ALL_TEST_FINAL3.groupby( ['Compte'])['Dif'].sum().reset_index()







display(ALL_TEST_FINAL3)


with pd.ExcelWriter(r'C:\\Users\\AROLMCAC\\Desktop\\Test python\\FINALTEST.xlsx') as writer:
    Result1.to_excel(writer, sheet_name='Test', index = False)
    FINALDF2.to_excel(writer, sheet_name='Pivot_TB',index= False)
    FINALPOSTINGS1.to_excel(writer, sheet_name='Pivot_Postings',index = False)
    FINALDF3.to_excel(writer, sheet_name='TB_BY_ACCOUNT',index = False)
    ALL_TEST_FINAL1.to_excel(writer, sheet_name='TEST_FINAL',index = False)
    ALL_TEST_FINAL3.to_excel(writer, sheet_name='SEE',index = False)

File names: ['C:\\Users\\AROLMCAC\\Desktop\\Test python\\Test\\AP UK CEGID BAL GEN 010121.xlsx', 'C:\\Users\\AROLMCAC\\Desktop\\Test python\\Test\\AP UK CEGID BAL GEN 300622.xlsx', 'C:\\Users\\AROLMCAC\\Desktop\\Test python\\Test\\AP UK CEGID BAL GEN 311221.xlsx']
Final result:
File names: ['C:\\Users\\AROLMCAC\\Desktop\\Test python\\Accounting_data\\COMPTA.xlsx']


C:\Temp\ipykernel_15872\20050611.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finalexcelsheet = finalexcelsheet.append(df, ignore_index=True)
C:\Temp\ipykernel_15872\20050611.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finalexcelsheet = finalexcelsheet.append(df, ignore_index=True)
C:\Temp\ipykernel_15872\20050611.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finalexcelsheet = finalexcelsheet.append(df, ignore_index=True)
C:\Temp\ipykernel_15872\20050611.py:37: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  FINALDF2 = FINALDF1.groupby( ['LEFT'])['Solde_2020','Solde_2021','Solde_2022'].sum().reset_index()
C:\Temp\ipykernel

,LEFT,Nature,Beginning Balance,Debit postings,Credit postings,Ending Balance by FFP,Ending Balance by Entity,Dif
0,1,BS,493799,325338,-349074,470063,205183,264880
1,2,BS,9388,184544,-149442,44490,44490,0
2,4,BS,-540791,7531778,-7043908,-52921,-15314,-37607
3,5,BS,0,328763,-329855,-1092,-1092,0
4,6,P&L,37600,1885738,-446292,1477046,653557,823489
5,7,P&L,0,978989,-2916580,-1937591,-886830,-1050761


C:\Temp\ipykernel_15872\20050611.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TEST_COMPTE['GENERAL'] = TEST_COMPTE['GENERAL'].astype(int)
C:\Temp\ipykernel_15872\20050611.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TEST_COMPTE.rename(columns = {'GENERAL':'Compte'}, inplace = True)
C:\Temp\ipykernel_15872\20050611.py:131: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ALL_TEST_FINAL1 = ALL_TEST_FINAL.groupby( ['Compte'])['Nature','Beginning Balance','Debit postings',

,Compte,Nature,Beginning Balance,Debit postings,Credit postings,Ending Balance by FFP,Ending Balance by Entity,Dif
0,40100000,BS,-80151,0.0,7348.35,-87499.35,0,-87499.35
1,40100000,BS,0,0.0,7348.35,-7348.35,-1543,-5805.35
2,40100000,BS,0,0.0,7348.35,-7348.35,0,-7348.35
3,40100000,BS,-80151,0.0,648.41,-80799.41,0,-80799.41
4,40100000,BS,0,0.0,648.41,-648.41,-1543,894.59
...,...,...,...,...,...,...,...,...
6760,42865500,BS,0,NaN,NaN,NaN,0,NaN
6761,45615000,BS,0,NaN,NaN,NaN,0,NaN
6762,48605500,BS,0,NaN,NaN,NaN,0,NaN
6763,62320000,P&L,0,NaN,NaN,NaN,0,NaN
